# [Размер случайного леса](https://www.coursera.org/learn/vvedenie-mashinnoe-obuchenie/programming/57ezh/razmier-sluchainogho-liesa)

## Введение
Случайный лес — это модель классификации, объединяющая некоторое количество решающих деревьев в одну композицию, за счет чего улучшается их качество работы и обобщающая способность. Деревья строятся независимо друг от друга. Чтобы они отличались друг от друга, обучение проводится не на всей обучающей выборке, а на ее случайном подмножестве. Также, для дальнейшего уменьшения схожести деревьев, оптимальный признак для разбиения выбирается не из всех возможных признаков, а лишь из их случайного подмножества. Прогнозы, выданные деревьями, объединяются в один ответ путем усреднения.

Особенность случайного леса заключается в том, что он не переобучается по мере увеличения количества деревьев в композиции. Это достигается за счет того, что деревья не зависят друг от друга, и поэтому добавление нового дерева в композицию не усложняет модель, а лишь понижает уровень шума в прогнозах.

---

В библиотеке scikit-learn случайные леса реализованы в классах sklearn.ensemble.RandomForestClassifier (для классификации) и sklearn.ensemble.RandomForestRegressor (для регрессии). Обучение модели производится с помощью функции fit, построение прогнозов — с помощью функции predict. Число деревьев задается с помощью поля класса n_estimators. 

## 1. Загрузите данные из файла abalone.csv. 
Это датасет, в котором требуется предсказать возраст ракушки (число колец) по физическим измерениям.

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

In [50]:
data = pd.read_csv('./data/abalone.csv')
data.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## 2. Преобразуйте признак Sex в числовой: значение F должно перейти в -1, I — в 0, M — в 1. 
Если вы используете Pandas, то подойдет следующий код:  
`data['Sex'] = data['Sex'].map(lambda x: 1 if x == 'M' else (-1 if x == 'F' else 0))`

In [51]:
data['Sex'] = data['Sex'].map(lambda x: 1 if x == 'M' else (-1 if x == 'F' else 0))
data.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,-1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## 3. Разделите содержимое файлов на признаки и целевую переменную. 
В последнем столбце записана целевая переменная, в остальных — признаки.

In [52]:
X = data.loc[:, :'ShellWeight']
y = data['Rings']

## 4. Обучите случайный лес (sklearn.ensemble.RandomForestRegressor) с различным числом деревьев: от 1 до 50 (не забудьте выставить "random_state=1" в конструкторе). 
Для каждого из вариантов оцените качество работы полученного леса на кросс-валидации по 5 блокам. Используйте параметры "random_state=1" и "shuffle=True" при создании генератора кросс-валидации sklearn.cross_validation.KFold. В качестве меры качества воспользуйтесь коэффициентом детерминации (sklearn.metrics.r2_score).

In [63]:
# clf = RandomForestRegressor(random_state=1)
clf = RandomForestRegressor()
cv = KFold(n_splits=5, shuffle=True, random_state=1)
grid = {'n_estimators': np.arange(1, 51)}
gs = GridSearchCV(clf, grid, scoring='r2', cv=cv, n_jobs=-1)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

## 5. Определите, при каком минимальном количестве деревьев случайный лес показывает качество на кросс-валидации выше 0.52. Это количество и будет ответом на задание.

In [64]:
for k, v in gs.cv_results_.items():
    if 'mean' in k:
        print(f"Key: {k}, value: {v}\n")

Key: mean_fit_time, value: [0.01857486 0.03546524 0.04888883 0.06467738 0.07981029 0.09571934
 0.11032181 0.12697501 0.14111667 0.15678477 0.17238841 0.18739681
 0.20286093 0.22077799 0.23375883 0.25058265 0.26529555 0.28138542
 0.2956028  0.31051106 0.32613106 0.34238687 0.35751448 0.37571187
 0.38763232 0.40405011 0.41825576 0.43584819 0.44976263 0.46780963
 0.4814055  0.49721708 0.51356249 0.52813559 0.54367933 0.55903797
 0.57347183 0.59111562 0.60591049 0.62002721 0.63582535 0.65090656
 0.66673222 0.68518052 0.69969945 0.71282334 0.73008981 0.74455276
 0.76135678 0.76251717]

Key: mean_score_time, value: [0.00196695 0.00220237 0.00240955 0.0026134  0.00287409 0.00308561
 0.00336175 0.00361042 0.00387511 0.00408292 0.00434761 0.00457683
 0.004915   0.00502706 0.00530415 0.0055377  0.00583591 0.00605974
 0.00633712 0.00654454 0.00678778 0.00854173 0.00733614 0.00751185
 0.00781641 0.00804691 0.00824571 0.00860872 0.00878143 0.00898786
 0.00924616 0.00947857 0.00972276 0.00992579 0.0

In [65]:
for i, val in enumerate(gs.cv_results_['mean_test_score']):
    if val > 0.52:
        print(i + 1)
        break
len(gs.cv_results_['mean_test_score'])     

21


50